In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as nps
import pykonal

In [ ]:
solver_c                      = pykonal.EikonalSolver()
solver_c.mode                 = 'cartesian'
solver_c.vgrid.min_coords     = -20, -20, 0
solver_c.vgrid.node_intervals = 1, 1, 1
solver_c.vgrid.npts           = 41, 41, 1
solver_c.vv                   = np.ones(solver_c.vgrid.npts)
solver_c.mode                 = solver_c.mode
solver_c.pgrid.min_coords     = solver_c.vgrid.min_coords
solver_c.pgrid.node_intervals = solver_c.vgrid.node_intervals
solver_c.pgrid.npts           = solver_c.vgrid.npts

isrc = 20, 20, 0
solver_c.uu[isrc]     = 0
solver_c.is_far[isrc] = False
solver_c.is_alive[isrc] = True
solver_c.close.append(isrc)
solver_c.solve()

In [ ]:
solver_s                      = pykonal.EikonalSolver()
solver_s.mode                 = 'spherical'
solver_s.vgrid.min_coords     = 0, np.pi/2, 0
solver_s.vgrid.node_intervals = 1, 1, np.pi/10
solver_s.vgrid.npts           = 20, 1, 21
solver_s.vv                   = np.ones(solver_s.vgrid.npts)
solver_s.mode                 = solver_s.mode
solver_s.pgrid.min_coords     = solver_s.vgrid.min_coords
solver_s.pgrid.node_intervals = solver_s.vgrid.node_intervals
solver_s.pgrid.npts           = solver_s.vgrid.npts


for ip in range(solver_s.pgrid.npts[2]):
    isrc = (0, 0, ip)
    solver_s.uu[isrc]       = 0
    solver_s.is_far[isrc]   = False
    solver_s.is_alive[isrc]   = True
    solver_s.close.append(isrc)
solver_s.solve()

In [ ]:
rr = np.sqrt(np.sum(np.square(solver_c.pgrid[...]), axis=-1))
tt = np.arccos(solver_c.pgrid[...,2] / rr)
tt[20, 20] = 0
pp = np.mod(np.arctan2(solver_c.pgrid[...,1], solver_c.pgrid[...,0]), 2 * np.pi)
rtp = np.moveaxis(np.stack([rr, tt, pp]), 0, -1)

def decorate(func):
    def wrapper(*args):
        try:
            return (func(*args))
        except Exception:
            return (np.inf)
    return (wrapper)

uui = np.apply_along_axis(
    decorate(pykonal.LinearInterpolator3D(solver_s.pgrid, solver_s.uu).interpolate),
    -1, 
    rtp
)
uui[20, 20, 0] = 0

In [ ]:
iz, it = 0, 0
size = 10
vmin, vmax = rr.min(), rr.max()
cmap = plt.get_cmap('jet_r')
plt.close('all')
fig = plt.figure(figsize=(6,8))

ax1 = fig.add_subplot(2, 2, 1, aspect=1)
ax1.set_title('Cartesian')
pts = ax1.scatter(
    solver_c.pgrid[:,:,iz,0],
    solver_c.pgrid[:,:,iz,1],
    c=solver_c.uu[:,:,iz],
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    s=size,
    linewidth=0
)


xx = solver_s.pgrid[:,it,:,0] * np.sin(solver_s.pgrid[:,it,:,1]) * np.cos(solver_s.pgrid[:,it,:,2])
yy = solver_s.pgrid[:,it,:,0] * np.sin(solver_s.pgrid[:,it,:,1]) * np.sin(solver_s.pgrid[:,it,:,2])
ax2 = fig.add_subplot(2, 2, 2, aspect=1)
ax2.set_title('Polar')
pts = ax2.scatter(
    xx,
    yy,
    c=solver_s.uu[:,it,:],
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    s=size,
    linewidth=0
)
cbar = fig.colorbar(pts, ax=(ax1, ax2), orientation='horizontal', shrink=0.5)
cbar.set_label('Travel time [s]')


ax3 = fig.add_subplot(2, 2, 3, aspect=1)
pts = ax3.scatter(
    solver_c.pgrid[:,:,iz,0],
    solver_c.pgrid[:,:,iz,1],
     c=np.abs((solver_c.uu[:,:,iz] - rr[:,:,iz])),
    cmap=cmap,
    s=size,
    linewidth=0
)

ax4 = fig.add_subplot(2, 2, 4, aspect=1)
pts = ax4.scatter(
    solver_c.pgrid[:,:,iz,0],
    solver_c.pgrid[:,:,iz,1],
    c=np.abs((uui[:,:,iz] - rr[:,:,iz])),
    cmap=cmap,
    vmin=pts.get_array().min(),
    vmax=pts.get_array().max(),
    s=size,
    linewidth=0
)
cbar = fig.colorbar(pts, ax=(ax3, ax4), orientation='horizontal', shrink=0.5)
cbar.set_label('Absolute error [s]')

for ax in (ax1, ax2, ax3, ax4):
    ax.set_xlim(solver_c.pgrid[..., 0].min(), solver_c.pgrid[..., 0].max())
    ax.set_ylim(solver_c.pgrid[..., 1].min(), solver_c.pgrid[..., 1].max())

In [ ]:
plt.savefig('/Users/malcolmwhite/Desktop/errors.png', bbox_inches='tight')